In [ ]:
def _convert_coordinates_to_raster(coords, img_size, xymax):
        # __author__ = visoft
        # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
        Xmax, Ymax = xymax
        H, W = img_size
        W1 = 1.0 * W * W / (W + 1)
        H1 = 1.0 * H * H / (H + 1)
        xf = W1 / Xmax
        yf = H1 / Ymax
        coords[:, 1] *= yf
        coords[:, 0] *= xf
        coords_int = np.round(coords).astype(np.int32)
        return coords_int


    def _get_xmax_ymin(self, grid_sizes_panda, imageId):
        # __author__ = visoft
        # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
        xmax, ymin = grid_sizes_panda[grid_sizes_panda.ImageId == imageId].iloc[0, 1:].astype(float)
        return (xmax, ymin)


    def _get_polygon_list(self, wkt_list_pandas, imageId, cType):
        # __author__ = visoft
        # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
        df_image = wkt_list_pandas[wkt_list_pandas.ImageId == imageId]
        multipoly_def = df_image[df_image.ClassType == cType].MultipolygonWKT
        polygonList = None
        if len(multipoly_def) > 0:
            assert len(multipoly_def) == 1
            polygonList = wkt_loads(multipoly_def.values[0])
        return polygonList


    def _get_and_convert_contours(self, polygonList, raster_img_size, xymax):
        # __author__ = visoft
        # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
        perim_list = []
        interior_list = []
        if polygonList is None:
            return None
        for k in range(len(polygonList)):
            poly = polygonList[k]
            perim = np.array(list(poly.exterior.coords))
            perim_c = _convert_coordinates_to_raster(perim, raster_img_size, xymax)
            perim_list.append(perim_c)
            for pi in poly.interiors:
                interior = np.array(list(pi.coords))
                interior_c = _convert_coordinates_to_raster(interior, raster_img_size, xymax)
                interior_list.append(interior_c)
        return perim_list, interior_list


    def _plot_mask_from_contours(self, raster_img_size, contours, class_value=1):
        # __author__ = visoft
        # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
        img_mask = np.zeros(raster_img_size, np.uint8)
        if contours is None:
            return img_mask
        perim_list, interior_list = contours
        cv2.fillPoly(img_mask, perim_list, class_value)
        cv2.fillPoly(img_mask, interior_list, 0)
        return img_mask


    def _generate_mask_for_image_and_class(self, raster_size, imageId, class_type):#grid_sizes_panda=GS, wkt_list_pandas=DF):
        # __author__ = visoft
        # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
        grid_sizes_panda = self.grid_sizes
        wkt_list_pandas = self.polygons
        xymax = _get_xmax_ymin(grid_sizes_panda, imageId)
        polygon_list = _get_polygon_list(wkt_list_pandas, imageId, class_type)
        contours = _get_and_convert_contours(polygon_list, raster_size, xymax)
        mask = _plot_mask_from_contours(raster_size, contours, 1)
        return mask

In [1]:
class MapObject (object):
    def __init__(img, polygons = None, poly_grids = None, mask = None):
        '''Creates object that includes img (heigth, width, channels), poligons - dictionary {class:list of coordinates}, corresponding mask (heigth, width, num_classes)'''
        self.img = img
        self.polygons = polygons
        self.mask = mask
        self.poly_grids = poly_grids
    
    def _plot_mask_from_contours(self, raster_img_size, contours, class_value=1):
        img_mask = np.zeros(raster_img_size, np.uint8)
        if contours is None:
            return img_mask
        perim_list, interior_list = contours
        cv2.fillPoly(img_mask, perim_list, class_value)
        cv2.fillPoly(img_mask, interior_list, 0)
        return img_mask
    
     def _get_and_convert_contours(self, polygonList, raster_img_size, xymax):
        perim_list = []
        interior_list = []
        if polygonList is None:
            return None
        for k in range(len(polygonList)):
            poly = polygonList[k]
            perim = np.array(list(poly.exterior.coords))
            perim_c = _convert_coordinates_to_raster(perim, raster_img_size, xymax)
            perim_list.append(perim_c)
            for pi in poly.interiors:
                interior = np.array(list(pi.coords))
                interior_c = _convert_coordinates_to_raster(interior, raster_img_size, xymax)
                interior_list.append(interior_c)
        return perim_list, interior_list
    
    def _convert_coordinates_to_raster(coords, img_size, xymax):
        Xmax, Ymax = xymax
        H, W = img_size
        W1 = 1.0 * W * W / (W + 1)
        H1 = 1.0 * H * H / (H + 1)
        xf = W1 / Xmax
        yf = H1 / Ymax
        coords[:, 1] *= yf
        coords[:, 0] *= xf
        coords_int = np.round(coords).astype(np.int32)
        return coords_int
    
    def _generate_mask_for_image_and_class(self, raster_size, class_type):
        contours = _get_and_convert_contours(polygon_list, raster_size, xymax)
        mask = _plot_mask_from_contours(raster_size, contours)
        return mask
    
    def poly2mask():
        mask_multi =[]
        _, h, w = img.shape
        for cls in self.polygons.ClassType.unique():
            mask = generate_mask_for_image_and_class((h, w), id, cls)
            mask_multi.append(mask)
        mask_multi = np.array(mask_multi)
        self.mask = mask_multi
        
    def mask2poly():
    
    def split():
    
    def _augmentation():
        
    def _crop():
        
    
    

IndentationError: expected an indented block (<ipython-input-1-d31c949ab140>, line 19)